## Functions

In [16]:
from LyndonWords import *
import itertools
import progressbar
import math
def genPermutations(type,n,k):
    rootSystemPermutations = np.empty(math.factorial(n+1),dtype=object)
    bar = progressbar.ProgressBar(maxval=math.factorial(n+1))
    v = 0
    for i in bar(itertools.permutations(list(range(n+1)))):
        rootSystemPermutations[v] = rootSystem(i,type,k)
        v+=1
    return rootSystemPermutations

In [17]:
def checkMonotoneIncreasingImpDeltaStart(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [i[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(str(i).startswith(str(j))):
                    flag = True
                    break
            if(flag):
                continue
            for j in rootsys.getAffineWords(i)[1:]:
                if np.any(np.all(j[0:sum(rootsys.delta)] == deltaArrays)):
                    returnArr.append((rootsys.ordering,j))
    return returnArr

In [18]:
def checkDeltaStartImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            flag = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaWeight] == i)):
                    flag = True
                    break
            if not flag:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [19]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [29]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    for i in rootSystem.getAffineWords(comb):
        if(k % rootSystem.n == 0):
            print(k //rootSystem.n)
        print(" " + str(i))
        k+=1

In [21]:
def exportByLength(rootSystem):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                file.write(str(i) + '\n')
    file.close()

## Type C

In [35]:
rootSystemPermutations = genPermutations('C',4,3)

  0% (0 of 120) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


  0% (1 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:19
  1% (2 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:18
  2% (3 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:17
  3% (4 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:17
  4% (5 of 120) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:17
  5% (6 of 120) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:17
  5% (7 of 120) |#                       | Elapsed Time: 0:00:01 ETA:   0:00:17
  6% (8 of 120) |#                       | Elapsed Time: 0:00:01 ETA:   0:00:16
  7% (9 of 120) |#                       | Elapsed Time: 0:00:01 ETA:   0:00:16
  8% (10 of 120) |#                      | Elapsed Time: 0:00:01 ETA:   0:00:16
  9% (11 of 120) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:16
 10% (12 of 120) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:16
 10% (13 of 120) |##                    

In [36]:
exceptions = checkMonotoneIncreasingImpDeltaStart(rootSystemPermutations)
if(len(exceptions) != 0):
    print("Exception")

Above shows that monotone increasing implies it starts with a delta word

In [37]:
checkDeltaStartImpMonotoneIncreasing(rootSystemPermutations)

[]

Implies that beginning with a k-delta word implies monotone increasing

In [38]:
arr = []
for r in range(n):
    arr.append([])
for rootsys in rootSystemPermutations:
    word = rootsys.getWords(2*rootsys.delta)[0]
    for i in range(1,len(word)):
        if(word[i] == rootsys.ordering[0]):
            arr[rootsys.deltaWeight-i-1].append(rootsys.ordering)
            break
for i in range(len(arr)):
    print("")
    print(i+1,end='\n ')
    for j in arr[i]:
        print(str(j),end='\n ')



1
 0<1<2<3<4
 0<1<2<4<3
 0<1<3<2<4
 0<1<3<4<2
 0<1<4<2<3
 0<1<4<3<2
 0<2<1<3<4
 0<2<1<4<3
 0<2<3<1<4
 0<2<3<4<1
 0<2<4<1<3
 0<2<4<3<1
 0<3<1<2<4
 0<3<1<4<2
 0<3<2<1<4
 0<3<2<4<1
 0<3<4<1<2
 0<3<4<2<1
 0<4<1<2<3
 0<4<1<3<2
 0<4<2<1<3
 0<4<2<3<1
 0<4<3<1<2
 0<4<3<2<1
 4<0<1<2<3
 4<0<1<3<2
 4<0<2<1<3
 4<0<2<3<1
 4<0<3<1<2
 4<0<3<2<1
 4<1<0<2<3
 4<1<0<3<2
 4<1<2<0<3
 4<1<2<3<0
 4<1<3<0<2
 4<1<3<2<0
 4<2<0<1<3
 4<2<0<3<1
 4<2<1<0<3
 4<2<1<3<0
 4<2<3<0<1
 4<2<3<1<0
 4<3<0<1<2
 4<3<0<2<1
 4<3<1<0<2
 4<3<1<2<0
 4<3<2<0<1
 4<3<2<1<0
 
2
 1<2<0<3<4
 1<2<0<4<3
 1<2<3<0<4
 1<2<3<4<0
 1<2<4<0<3
 1<2<4<3<0
 1<3<2<0<4
 1<3<2<4<0
 1<3<4<2<0
 1<4<2<0<3
 1<4<2<3<0
 1<4<3<2<0
 3<0<1<2<4
 3<0<2<1<4
 3<0<2<4<1
 3<1<0<2<4
 3<1<2<0<4
 3<1<2<4<0
 3<2<0<1<4
 3<2<0<4<1
 3<2<1<0<4
 3<2<1<4<0
 3<2<4<0<1
 3<2<4<1<0
 
3
 1<3<0<2<4
 1<3<0<4<2
 1<3<4<0<2
 1<4<3<0<2
 2<0<1<3<4
 2<0<1<4<3
 2<1<0<3<4
 2<1<0<4<3
 2<1<3<0<4
 2<1<3<4<0
 2<1<4<0<3
 2<1<4<3<0
 2<3<0<1<4
 2<3<0<4<1
 2<3<1<0<4
 2<3<1<4<0
 2<3<4<0<1
 2<3<4<1<0

Starts with 0< n, n< 0: 1

i< i+1: i+1

i< i-1: n-(i-1)

The prefix of SL_1(\delta) up to and including the smallest letter gets appended to the word similarly to in the standard order for SL_1(k\delta).

SL_i(k\delta) works exactly the same as in SL_i(k\delta) for the standard order, except the word ends with whatever SL(\delta) cuts off instead of just {r-1}

## Type G

In [22]:
G2perms = genPermutations('G',2,5)

  0% (0 of 6) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 16% (1 of 6) |####                      | Elapsed Time: 0:00:00 ETA:   0:00:00
 33% (2 of 6) |########                  | Elapsed Time: 0:00:00 ETA:   0:00:00
 50% (3 of 6) |#############             | Elapsed Time: 0:00:00 ETA:   0:00:00
 66% (4 of 6) |#################         | Elapsed Time: 0:00:00 ETA:   0:00:00
 83% (5 of 6) |#####################     | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (6 of 6) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [23]:
checkDeltaStartImpMonotoneIncreasing(G2perms)

[]

In [24]:
checkMonotoneIncreasingImpDeltaStart(G2perms)

[]

In [25]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [43]:
index = 4
print(str(G2perms[index].ordering))

printByBaseWord(G2perms[index],G2perms[0].delta)

2<0<1
0
 2,2,1,0,2,1
 2,2,1,2,1,0
1
 2,2,1,0,2,2,1,0,2,1,2,1
 2,2,1,2,1,2,2,1,0,2,1,0
2
 2,2,1,0,2,2,1,0,2,1,2,2,1,0,2,1,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
3
 2,2,1,0,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
4
 2,2,1,0,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
5
 2,2,1,0,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
